In [8]:
import os
import numpy as np
import matplotlib as mpl
import pickle
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import scipy.io as spio

def load_and_convert_mat(filename):
    _check_keys = lambda dict_: {key: _todict(val) if isinstance(val, spio.matlab.mio5_params.mat_struct) else val for key, val in dict_.items()}
    _todict = lambda matobj: {strg: _todict(getattr(matobj, strg)) if isinstance(getattr(matobj, strg), spio.matlab.mio5_params.mat_struct) else getattr(matobj, strg) for strg in matobj._fieldnames}
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def load_and_process_data(file_path):
    file = load_and_convert_mat(file_path)
    data = {'S': {}}
    data['S']['day1'] = file['analysis_struct'][0]
    data['S']['day3'] = file['analysis_struct'][1]
    data['S']['day3am'] = file['analysis_struct'][2]
    data['S']['day5'] = file['analysis_struct'][3]
    data['S']['day5am'] = file['analysis_struct'][4]
    data['S']['pre'] = file['analysis_struct'][6]
    data['S']['imm'] = file['analysis_struct'][5]
    data['S']['training'] = file['analysis_struct'][8]
    data['S']['recent'] = file['analysis_struct'][7]
    for ndarray in data['S']:
        data['S'][ndarray] = np.vstack(data['S'][ndarray])
    return data

merge = False
if merge:
    sessions = ["Pre-120h", "Pre-72h", "Pre-24h", "Pre-3h + Imm", "Training"]  
else:
    sessions = ["Pre-120h", "Pre-72h AM", "Pre-72h PM", "Pre-24h AM", "Pre-24h PM", "Pre-3h", "Imm", "Training"]

In [ ]:
# Load your MATLAB data here
file_paths = [
    "/Volumes/LaCie/Mahe/baseline+day1/analysis_of_038-01.mat",
    "/Volumes/LaCie/Mahe/baseline+day1/analysis_of_039-02.mat",
    "/Volumes/LaCie/Mahe/baseline+day1/analysis_of_038-04.mat",
    "/Volumes/LaCie/Mahe/baseline+day1/analysis_of_038-03.mat",
]
mice = [load_and_process_data(file_path) for file_path in file_paths]

In [10]:
ref_sess = "recent"
eng_thresh = 0
sess_list = ["day1", "day3am", "day3", "day5am", "day5", "pre", "imm", "training"]
results = {"tr": [], "tr_rel":[], "eng":[], "sess": [], "mouse": []}

for mm, mouse in enumerate(mice):   
    tr_ref = np.mean(mouse['S'][ref_sess]>0, axis=1)
    active_ref = tr_ref > 0
    tr_ref[active_ref] = (tr_ref[active_ref] - np.mean(tr_ref[active_ref]))/np.std(tr_ref[active_ref])

    for ss, sess in enumerate(sess_list):
        tr_sess = np.mean(mouse['S'][sess]>0, axis=1)
        active_sess = tr_sess > 0
        tr_rel = (tr_sess - np.mean(tr_sess[active_sess])) / np.std(tr_sess[active_sess])
        # Adjust based on sessions that merges
        if merge:
            split = False
            # Merge Sessions
            if ss == 1:
                ss = 2
                split = True
            if ss == 3:
                ss = 4
                split = True
            if ss == 5:
                ss = 6
                split = True
            # Adjust Session Numbers
            if ss == 2:
                ss = 1 
            elif ss == 4:
                ss = 2
            elif ss == 6:
                ss = 3
            elif ss == 7:
                ss = 4
        
        if split:
            results["mouse"].extend((mm+5)*np.ones(np.sum(active_sess)))
        else:
            results["mouse"].extend((mm)*np.ones(np.sum(active_sess)))
            
        results["tr"].extend(list(tr_sess[active_sess]))
        results["tr_rel"].extend(list(tr_rel[active_sess]))
        results["eng"].extend(list(tr_ref[active_sess] > eng_thresh))
        results["sess"].extend(ss*np.ones(np.sum(active_sess)))

In [ ]:
# Plot numbers of engrams and non-engrams
engrams = {x:0 for x in range(len(sessions))}
non_engrams = engrams.copy()
for j in range(len(sessions)):
    for i in range(len(results["eng"])):
        if results["sess"][i] == j:
            if results["eng"][i] == True:
                engrams[j] += 1
            else:
                non_engrams[j] += 1
gph = {x: engrams[x]/non_engrams[x] for x in list(engrams.keys())}
plt.figure().set_figwidth(10) 
plt.bar(range(len(non_engrams)), list(non_engrams.values()), color='gray')
plt.bar(range(len(engrams)), list(engrams.values()), color='orange')
plt.xticks(range(len(sessions)), sessions)
plt.ylabel("Number of Engrams and Non-Engrams")
plt.xlabel("Session")

In [ ]:
# Plot Engram Non Engram Ratio
data = {"mouse":[], "session":[], "ratio":[]}

for mouse in range(len(file_paths)):
    for a in range(len(sessions)):
        eng = 0
        noneng = 0
        for b in range(len(results["eng"])):
            if results["mouse"][b] == mouse:
                if results["sess"][b] == a:
                    if results["eng"][b] == True:
                        eng += 1
                    else:
                        noneng += 1
        data["mouse"].append(mouse)
        data["session"].append(a)
        data["ratio"].append(eng/noneng)

dt = pd.DataFrame(data).groupby(["mouse", "session"]).mean().reset_index()
plt.figure().set_figwidth(10)                                                
sns.boxplot(x="session", y="ratio", data=dt)
sns.stripplot(x="session", y="ratio", data=dt) # uncomment this if you want to plot individual data points
plt.ylabel("Engram/Non-Engram Ratio")
plt.xlabel("Session")
#plt.ylim([-0.1, 0.5])
sessions += [" "]
plt.xticks(np.arange(len(sessions)), sessions)
plt.tight_layout()
plt.legend([])

In [ ]:
# Plot Relative Transient Rate
# Uncomment the groupby section to see the results grouped by mouse 

df = pd.DataFrame(results).groupby(["mouse", "sess", "eng"]).mean().reset_index()                                               
sns.catplot(x="sess", y="tr_rel", hue="eng", data=df, kind="point", ci=68, palette=["gray", "royalblue"], capsize=0.2, scale=1., errwidth=3.5,  join=True, aspect=1.5)
sns.stripplot(x="sess", y="tr_rel", hue="eng", data=df)                            # uncomment this if you want to plot individual data points
plt.ylabel("Relative transient rate (z-score)")
plt.xlabel("Session")
# plt.ylim([-0.2, 1])
plt.xticks(np.arange(len(sessions)), sessions)
plt.tight_layout()
plt.legend([])